# Flux Balance Analysis

## FBA

FreeFlux is capable of solving the canonical [flux balance analysis](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3108565/) (FBA) problem, which is defined as:

\begin{aligned}
\max \quad & {{\bf{c}}^T} \cdot {\bf{v}}\\
s.t. \quad & {\bf{S}} \cdot {\bf{v}} = {\bf{0}}\\
&{{\bf{v}}_{lb}} \le {\bf{v}} \le {{\bf{v}}_{ub}}\\
\end{aligned}

where ${\bf{S}}$ is the stoichiometric matrix, and ${\bf{v}}$ is the flux vector bounded by ${{\bf{v}}_{lb}}$ and ${{\bf{v}}_{ub}}$. The objective function could be the linear combination of fluxes, and usually biomass formation is used.

Here we use the *E. coli* network for example. The model file can be found [here](https://github.com/Chaowu88/freeflux/tree/main/models/ecoli).

In [1]:
from freeflux import Model

MODEL_FILE = 'path/to/reactions.xlsx'

model = Model('ecoli')
model.read_from_file(MODEL_FILE)

with model.optimizer() as opt:
    # set bounds for fluxes
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])

    # solve the FBA problem
    opt.prepare()
    res = opt.optimize(objective = {'biom': 1})
print(res.opt_objective)

glycolysis_enzymes = ['pgi', 'pfk','fba', 'tpi', 'gapdh', 'eno', 'pk']
for enzyme in glycolysis_enzymes:
    print(enzyme, res.opt_fluxes[enzyme])


optimizing [elapsed: 0:00:01]

1.237
pgi 9.74646920121454
pfk 8.76944807419173
fba 8.76944807419176
tpi 8.76944807419176
gapdh 16.7783037520293
eno 14.6245286979626
pk 10.8080928072441


## FVA

[flux variable analysis](https://pubmed.ncbi.nlm.nih.gov/20920235/) (FVA) is used to find the minimum and maximum of a flux while maintaining objective above some fraction of its optimized value in the original FBA problem. The problem is formulated as: 

\begin{aligned}
\max/min \quad & {\rm{ }}{v_i}\\
s.t. \quad & {\bf{S}} \cdot {\bf{v}} = {\bf{0}}\\
&{{\bf{c}}^T} \cdot {\bf{v}} \ge \gamma  \cdot ob{j_{FBA}}\\
&{{\bf{v}}_{lb}} \le {\bf{v}} \le {{\bf{v}}_{ub}}\\
\end{aligned}

where $\gamma$ controls how close the FBA objective should be to its optiaml value $ob{j_{FBA}}$. 

FreeFlux includes FVA because it can provide a reasonable feasible space for fluxes and initial guesses sampled from it during the least squares optimzation. As an example, the flux ranges of the *E. coli* model can be estimated using the following lines:

In [2]:
with model.optimizer() as opt:
    # set bounds for fluxes
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])

    # estimate flux ranges
    opt.prepare()
    res = opt.estimate_fluxes_range(objective = {'biom': 1}, gamma = 0)

for enzyme in glycolysis_enzymes:
    print(enzyme, res.flux_ranges[enzyme])


estimating flux ranges [elapsed: 0:00:08]

pgi [-18.9459449212001, 9.99999999999999]
pfk [0.0, 10.0000000000007]
fba [0.0, 10.0000000000007]
tpi [0.0, 10.0000000000007]
gapdh [8.054955585922, 20.0000000000015]
eno [5.95104574299147, 20.0000000000016]
pk [0.0, 26.4203047242819]
